In [39]:
import os
from random import shuffle
import nibabel as nib
import numpy as np
from skimage.transform import resize
import torch

In [35]:
patients_path='D:/downloads/Resources/data'
patients = sorted(os.listdir(patients_path))
datas=[]
labels=[]

for patient_index in range(len(patients)):
    patient_path=os.path.join(patients_path,patients[patient_index])
    files=os.listdir(patient_path)
    filtered_files = [file for file in files if '_gt.nii.gz' in file]
    filtered_files = [file.replace('_gt.nii.gz','') for file in filtered_files]
    for i in range(2):
        image=nib.load(os.path.join(patient_path,filtered_files[0]+'.nii.gz'))
        image=image.get_fdata()
        mask=nib.load(os.path.join(patient_path,filtered_files[0]+'_gt.nii.gz'))
        mask=mask.get_fdata()
        image=image.astype(np.float32)
        image=np.divide((image-np.mean(image)),np.std(image))
        slice_num=image.shape[2]
        image = resize(image, (256,256,slice_num), mode='constant', cval=0)
        mask = resize(mask, (256,256,slice_num), mode='constant', cval=0)
        mask_new=np.zeros((256,256,slice_num,4))
        for idx in range(4):
            mask_new[:,:,:,idx]=mask[:,:,:]==idx
        for i in range(slice_num):
            datas.append(image[:,:,i])
            labels.append(mask_new[:,:,i,:])

idxs=np.array(range(2858))
shuffle(idxs)
train_group=idxs[0:2286]
val_group=idxs[2286:2572]
test_group=idxs[2572:2858]

train_data=torch.tensor(datas[train_group])
train_label=torch.tensor(labels[train_group])
val_data=torch.tensor(datas[val_group])
val_label=torch.tensor(labels[val_group])
test_data=torch.tensor(datas[test_group])
test_label=torch.tensor(labels[test_group])       




        #     softmax_out = outputs["pred_masks"]
        #     softmax_out = softmax_out.detach().cpu().numpy()
        #     prediction_cropped = np.squeeze(softmax_out[0,...])

        #     slice_predictions = np.zeros((4,x,y))
        #     # insert cropped region into original image again
        #     if x > nx and y > ny:
        #         slice_predictions[:,x_s:x_s+nx, y_s:y_s+ny] = prediction_cropped
        #     else:
        #         if x <= nx and y > ny:
        #             slice_predictions[:,:, y_s:y_s+ny] = prediction_cropped[:,x_c:x_c+ x, :]
        #         elif x > nx and y <= ny:
        #             slice_predictions[:,x_s:x_s + nx, :] = prediction_cropped[:,:, y_c:y_c + y]
        #         else:
        #             slice_predictions[:,:, :] = prediction_cropped[:,x_c:x_c+ x, y_c:y_c + y]
        #     prediction = transform.resize(slice_predictions,
        #                         (4, mask.shape[0], mask.shape[1]),
        #                         order=1,
        #                         preserve_range=True,
        #                         anti_aliasing=True,
        #                         mode='constant')
        #     prediction = np.uint8(np.argmax(prediction, axis=0))
        #     #prediction = keep_largest_connected_components(prediction)
        #     predictions.append(prediction)
        # prediction_arr = np.transpose(np.asarray(predictions, dtype=np.uint8), (1,2,0))
        # dir_pred = os.path.join(output_folder, "predictions")
        # makefolder(dir_pred)
        # out_file_name = os.path.join(dir_pred, label_file)
        # out_affine = mask_dat[1]
        # out_header = mask_dat[2]

        # save_nii(out_file_name, prediction_arr, out_affine, out_header)

        # dir_gt = os.path.join(output_folder, "masks")
        # makefolder(dir_gt)
        # mask_file_name = os.path.join(dir_gt, label_file)
        # save_nii(mask_file_name, mask_dat[0], out_affine, out_header)